In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np
import io

In [2]:
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/github-status/2019/20199' #subset of above
root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/github-status*' # all directories

In [3]:
def extract_html_data(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    result = []
    
    try:
        components = []
        for name in soup.find_all('span', {'class':'name'}):
            components.append(' '.join(name.text.split()))
        statuses = []
        for status in soup.find_all('span', {'class':'component-status'}):
            statuses.append(' '.join(status.text.split()))
    except:
        result.append('?')
    
    for x in range(len(statuses)):
        result.append([components[x], statuses[x]])
    
    return [result]

In [4]:
def read_file(file):
    source = file.split('/')[4]
    archive = ZipFile(file, 'r')
    namelist = archive.namelist()
    ret = []
    for item in namelist:
        if 'html' in item and 'history' not in item:
            html = archive.read(item)
            service = item.split('/')[1].split('.')[0]
            ts = item.split('/')[0]
            data = extract_html_data(html)
            data.insert(0, ts)
            data.insert(0, service)
            data.insert(0, source)
            ret.append(data)
    archive.close()
    return(ret)

In [5]:
files = glob.glob(root_dir + '/**/*.zip', recursive=True)
pool = mp.Pool(mp.cpu_count())
start_time = time.time()
try:
    results = pool.map(read_file, [file for file in files])
except:
    print('Error')
pool.close()
end_time = time.time()

In [6]:
print(end_time - start_time)

50.85820770263672


In [7]:
res = []
for outer in results:
    for inner in outer:
        res.append(inner)
print(len(res))

9612


In [8]:
column_names = ['source','service','timestamp','status']
df = pd.DataFrame(res, columns=column_names)
df.head(50)

,source,service,timestamp,status
0,github-status,global-status,20200508T150001,"[[Git Operations, Operational], [API Requests,..."
1,github-status,global-status,20200508T060001,"[[Git Operations, Operational], [API Requests,..."
2,github-status,global-status,20200508T130001,"[[Git Operations, Operational], [API Requests,..."
3,github-status,global-status,20200508T200001,"[[Git Operations, Operational], [API Requests,..."
4,github-status,global-status,20200508T030001,"[[Git Operations, Operational], [API Requests,..."
5,github-status,global-status,20200508T090001,"[[Git Operations, Operational], [API Requests,..."
6,github-status,global-status,20200508T180001,"[[Git Operations, Operational], [API Requests,..."
7,github-status,global-status,20200508T220001,"[[Git Operations, Operational], [API Requests,..."
8,github-status,global-status,20200508T010001,"[[Git Operations, Operational], [API Requests,..."
9,github-status,global-status,20200508T000001,"[[Git Operations, Operational], [API Requests,..."


In [10]:
df.to_csv(r'/home/shane/Documents/thesis/output/parsed/github-status.csv', index=False)